# Explore implementation of FCN model

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import tensorflow_datasets as tfds

import src.data.datasets.deep_globe_2018

2024-01-22 20:13:22.398213: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Reuse code from previous notebooks

In [2]:
IMAGE_SIZE = 224
BATCH_SIZE = 1

In [3]:
def normalize(input_image):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  return input_image

In [4]:
def rgb_to_index(image):
    palette = [
        [0, 255, 255],   # urban_land
        [255, 255, 0],   # agriculture_land
        [255, 0, 255],   # rangeland
        [0, 255, 0],     # forest_land
        [0, 0, 255],     # water
        [255, 255, 255], # barren_land
        [0, 0, 0]        # unknown
    ]
    
    one_hot_map = []
    for colour in palette:
        class_map = tf.reduce_all(tf.equal(image, colour), axis=-1)
        one_hot_map.append(class_map)
    one_hot_map = tf.stack(one_hot_map, axis=-1)
    one_hot_map = tf.cast(one_hot_map, tf.uint8)
    indexed = tf.math.argmax(one_hot_map, axis=2)
    indexed = tf.cast(indexed, dtype=tf.uint8)
    indexed = tf.expand_dims(indexed, -1)

    return indexed

In [5]:
def load_images(datapoint, image_size):
    
    images = tf.image.resize(datapoint['image'], (image_size, image_size))

    annotations = tf.map_fn(rgb_to_index, datapoint['segmentation_mask'])
    annotations = tf.image.resize(annotations, (image_size, image_size), method='nearest')
    
    images = normalize(images)

    return images, annotations

In [6]:
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    print(len(display_list))
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

In [7]:
(ds_train, ds_valid, ds_test), ds_info = tfds.load(
    name='deep_globe_2018',
    download=False,
    with_info=True,
    split=['all_images[700:710]', 'all_images[7:9]', 'all_images[9:10]']
)
train_batches = (
    ds_train
    .batch(BATCH_SIZE)
    .map(lambda x: load_images(x, IMAGE_SIZE), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)
validation_batches = (
    ds_valid
    .batch(BATCH_SIZE)
    .map(lambda x: load_images(x, IMAGE_SIZE), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)
test_batches = (
    ds_test
    .batch(BATCH_SIZE)
    .map(lambda x: load_images(x, IMAGE_SIZE), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)    

2024-01-22 20:13:25.652678: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


## Explore model

In [8]:
base_model = tf.keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
)

In [13]:
# Layers that need to be "fully convolutionalized" (layer sizes as per https://doi.org/10.48550/arXiv.1409.1556, last accessed: 22 Jan 2024)
# Conversion as per: https://cs231n.github.io/convolutional-networks/#convert, last accessed: 22 Jan 2024

# FC-4096
block_6_conv1 = tf.keras.layers.Conv2D(
    filters=4096,
    kernel_size=(7,7),
    strides=(1,1),
    padding='same',
    activation='relu'
)

# FC-4096
block_6_conv2 = tf.keras.layers.Conv2D(
    filters=4096,
    kernel_size=(7,7),
    strides=(1,1),
    padding='same',
    activation='relu'
)

# FC-1000 --> This final classifier layer is discarded
# DO NOTHING

# We append a 1 × 1 convolution with channel dimension 21 to predict scores for each of the PASCAL classes at each of the coarse output locations
# Looking at the original implementation there is no explicit activation function: https://github.com/shelhamer/fcn.berkeleyvision.org/blob/master/voc-fcn32s/net.py, last accessed 22.01.2024
# Hence "linear" is chosen
block_6_conv3 = tf.keras.layers.Conv2D(
    filters=7,
    kernel_size=(1,1),
    strides=(1,1),
    padding='same',
    activation='linear'
)

# followed by a deconvolution layer to bi-linearly upsample the coarse outputs to pixel-dense outputs
block_6_deconv1 = tf.keras.layers.Conv2DTranspose(
    filters=7,
    kernel_size=(64, 64),
    strides=(32, 32),
    use_bias=False, # As per original implementation
    padding='same',
    activation='softmax'
)
block_6_deconv1.trainable = False # As per original implementation: param=[dict(lr_mult=0)]






In [14]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = block_6_conv1(x)
x = block_6_conv2(x)
x = block_6_conv3(x) #scoring layer
x = block_6_deconv1(x)

2024-01-22 20:18:40.004733: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (mklcpu) ran out of memory trying to allocate 3.06GiB (rounded to 3288334336)requested by op StatelessRandomUniformV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-01-22 20:18:40.004804: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for mklcpu
2024-01-22 20:18:40.004820: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-01-22 20:18:40.004830: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-01-22 20:18:40.004839: I tensorflow/tsl/framework/bfc_allocator.cc:1046

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[7,7,4096,4096] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:StatelessRandomUniformV2] name: 

In [10]:
for i, m in train_batches.take(1):
    y = base_model(i)

In [11]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     